In [8]:
# Move up one level in directory to allow imports
import sys
sys.path.insert(0, '..')

import pandas as pd
from data.airport_dicts import *

In [48]:
# Import dataframes from static tables
average_fare_df = pd.read_csv("../data/table.csv")
la_seattle_df = pd.read_csv("../data/la_to_seattle_t1_static.csv")

# Define airports to select
market_airports = ["SEA", "SNA"]
market_coterms = ["LAX", "LGB", "BUR", "ONT", "PAE"]
market_all_airports = market_airports + market_coterms

# Filter for selected airports
average_fare_df = average_fare_df[average_fare_df["Airport Code"].isin(market_all_airports)]

# Drop state column
average_fare_df.drop('State Name', axis=1)

# Drop inflation column
average_fare_df.drop('Inflation Adjusted Average Fare ($) (Base Quarter:  Q2-2025)', axis=1)

average_fare_df

,2024 Passenger Rank,Airport Code,City Name,State Name,Average Fare ($),Inflation Adjusted Average Fare ($) (Base Quarter: Q2-2025)
0,1,LAX,Los Angeles,CA,413.68,413.68
7,8,SEA,Seattle,WA,400.25,400.25
46,47,SNA,Santa Ana,CA,406.95,406.95
52,53,ONT,Ontario,CA,375.80,375.80
54,55,BUR,Burbank,CA,316.68,316.68
73,74,LGB,Long Beach,CA,345.59,345.59
158,159,PAE,Everett,WA,381.42,381.42
